# Setup Environment

In [ ]:
# To run locally, uncomment the below
# ! pip install -e ../

# To run in Onyx, uncomment the below
import sys
sys.path.append("..")

from onyxgenai.embed import EmbeddingClient
from onyxgenai.model import ModelClient

embedding_client = EmbeddingClient("http://embed.onyx-services")
model_client = ModelClient("http://store.onyx-services")

# Deploy Embedding Model

In [ ]:
embedding_model = model_client.deploy_model("all-MiniLM-L6-v2", 1, 2, {"num_cpus": 2, "memory": 8000 * 1024 * 1024})

# Deploy LLM

In [ ]:
language_model = model_client.deploy_model("Mistral-7B-Instruct-v0.3", 1, 1, {})

# Embed Prompt for Vector Search

In [ ]:
query = "What is the capital of France?"
data = [query]
embeddings = model_client.embed_text(data, embedding_model)

# Perform Vector Search

In [ ]:
vector_data = embedding_client.vector_search(embeddings, "test_collection")
print(vector_data)

# Execute a RAG Search

In [ ]:
# Append RAG Data for Knowledge
context = ""
if vector_data:
    for data in vector_data:
        context = context + data["payload"]["text"]

prompt = f"""
Context Information:
---------------------
{context}
---------------------
Answer the following question using only the context information. If the question cannot be answered, say so.
Query: {query}
Answer:
"""  # noqa: E501

answer = model_client.generate_completion(prompt, model_name="Mistral-7B-Instruct-v0.3")
print(answer)

# Delete Model Deployments

In [ ]:
model_client.delete_deployment(embedding_model)
model_client.delete_deployment(language_model)